In [1]:
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px 
from plotly.offline import download_plotlyjs,init_notebook_mode, iplot
import plotly.tools as tls 
import plotly.figure_factory as ff
import folium
from folium.plugins import MarkerCluster

In [2]:
districts_geodata = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTBhWlGxCuVy8C9qAeXnkD8Kz_3AeDte4me63yESz1yqdpTnfUC2esXurQ_uZpxYxM6UoPzJbSZ_0yB/pub?gid=0&single=true&output=csv')

In [3]:
districts_geodata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   District   736 non-null    object 
 1   Latitude   736 non-null    float64
 2   Longitude  736 non-null    float64
dtypes: float64(2), object(1)
memory usage: 17.4+ KB


In [4]:
districts_geodata

,District,Latitude,Longitude
0,North and Middle Andaman,12.560980,92.817829
1,South Andaman,11.740087,92.658640
2,Anantapur,14.681888,77.600591
3,Chittoor,13.217176,79.100329
4,East Godavari,17.321250,82.040714
...,...,...,...
731,Purba Bardhaman,23.232421,87.861479
732,Purba Medinipur,21.937288,87.776333
733,Purulia,23.332203,86.361641
734,South 24 Parganas,22.135238,88.401604


In [5]:
districts_geodata.columns.values

array(['District', 'Latitude ', 'Longitude'], dtype=object)

In [6]:
index_names = districts_geodata[ districts_geodata['Latitude '] == 0 ].index 
districts_geodata.drop(index_names, inplace=True)

In [7]:
districts_geodata

,District,Latitude,Longitude
0,North and Middle Andaman,12.560980,92.817829
1,South Andaman,11.740087,92.658640
2,Anantapur,14.681888,77.600591
3,Chittoor,13.217176,79.100329
4,East Godavari,17.321250,82.040714
...,...,...,...
731,Purba Bardhaman,23.232421,87.861479
732,Purba Medinipur,21.937288,87.776333
733,Purulia,23.332203,86.361641
734,South 24 Parganas,22.135238,88.401604


In [8]:
#Covid-19 dataset wrt India. This dataset is being maintained by https://www.covid19india.org/
CovidDistrictData = pd.read_csv('https://api.covid19india.org/csv/latest/district_wise.csv')
CovidDistrictData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SlNo             801 non-null    int64  
 1   State_Code       801 non-null    object 
 2   State            801 non-null    object 
 3   District_Key     801 non-null    object 
 4   District         801 non-null    object 
 5   Confirmed        801 non-null    int64  
 6   Active           801 non-null    int64  
 7   Recovered        801 non-null    int64  
 8   Deceased         801 non-null    int64  
 9   Migrated_Other   801 non-null    int64  
 10  Delta_Confirmed  801 non-null    int64  
 11  Delta_Active     801 non-null    int64  
 12  Delta_Recovered  801 non-null    int64  
 13  Delta_Deceased   801 non-null    int64  
 14  District_Notes   66 non-null     object 
 15  Last_Updated     0 non-null      float64
dtypes: float64(1), int64(10), object(5)
memory usage: 100.2+ KB


In [9]:
CovidDistrictData.head()

,SlNo,State_Code,State,District_Key,District,Confirmed,Active,Recovered,Deceased,Migrated_Other,Delta_Confirmed,Delta_Active,Delta_Recovered,Delta_Deceased,District_Notes,Last_Updated
0,0,UN,State Unassigned,UN_Unassigned,Unassigned,0,0,0,0,0,0,0,0,0,NaN,NaN
1,1,AN,Andaman and Nicobar Islands,AN_Nicobars,Nicobars,0,0,0,0,0,0,0,0,0,District-wise numbers are out-dated as cumulat...,NaN
2,2,AN,Andaman and Nicobar Islands,AN_North and Middle Andaman,North and Middle Andaman,1,0,1,0,0,0,0,0,0,District-wise numbers are out-dated as cumulat...,NaN
3,3,AN,Andaman and Nicobar Islands,AN_South Andaman,South Andaman,51,19,32,0,0,0,0,0,0,District-wise numbers are out-dated as cumulat...,NaN
4,0,AP,Andhra Pradesh,AP_Foreign Evacuees,Foreign Evacuees,434,0,434,0,0,0,0,0,0,NaN,NaN


In [10]:
DistrictData = CovidDistrictData[['District','Confirmed','Recovered','Deceased','Active']].reset_index()
DistrictData.head()

,index,District,Confirmed,Recovered,Deceased,Active
0,0,Unassigned,0,0,0,0
1,1,Nicobars,0,0,0,0
2,2,North and Middle Andaman,1,1,0,0
3,3,South Andaman,51,32,0,19
4,4,Foreign Evacuees,434,434,0,0


In [11]:
merge_dfs = pd.merge(DistrictData, districts_geodata, on=['District'], how='inner')
merge_dfs.head()

,index,District,Confirmed,Recovered,Deceased,Active,Latitude,Longitude
0,2,North and Middle Andaman,1,1,0,0,12.560980,92.817829
1,3,South Andaman,51,32,0,19,11.740087,92.658640
2,4,Foreign Evacuees,434,434,0,0,19.664062,78.532011
3,211,Foreign Evacuees,0,0,0,0,19.664062,78.532011
4,570,Foreign Evacuees,33,0,0,33,19.664062,78.532011


In [12]:
merge_dfs = merge_dfs.drop(columns=['index'])
merge_dfs.head()

,District,Confirmed,Recovered,Deceased,Active,Latitude,Longitude
0,North and Middle Andaman,1,1,0,0,12.560980,92.817829
1,South Andaman,51,32,0,19,11.740087,92.658640
2,Foreign Evacuees,434,434,0,0,19.664062,78.532011
3,Foreign Evacuees,0,0,0,0,19.664062,78.532011
4,Foreign Evacuees,33,0,0,33,19.664062,78.532011


In [13]:
merge_dfs

,District,Confirmed,Recovered,Deceased,Active,Latitude,Longitude
0,North and Middle Andaman,1,1,0,0,12.560980,92.817829
1,South Andaman,51,32,0,19,11.740087,92.658640
2,Foreign Evacuees,434,434,0,0,19.664062,78.532011
3,Foreign Evacuees,0,0,0,0,19.664062,78.532011
4,Foreign Evacuees,33,0,0,33,19.664062,78.532011
...,...,...,...,...,...,...,...
800,Purba Bardhaman,7806,6548,73,1185,23.232421,87.861479
801,Purba Medinipur,14935,13235,192,1508,21.937288,87.776333
802,Purulia,5190,4613,28,549,23.332203,86.361641
803,South 24 Parganas,24663,21461,460,2742,22.135238,88.401604


In [14]:
loc = merge_dfs[['Latitude ', 'Longitude']]
loc_list = loc.values.tolist()
loc_list[:5]

[[12.5609802, 92.8178291],
 [11.740086699999999, 92.6586401],
 [19.6640624, 78.5320107],
 [19.6640624, 78.5320107],
 [19.6640624, 78.5320107]]

## Daily Update

In [15]:
# Make an empty map
ConfirmedCasesMap = folium.Map(location=[29.742316, 76.219202], tiles='OpenStreetMap', zoom_start=5)
 
# I can add marker one by one on the map
for i in range(0,len(merge_dfs)):
    folium.Circle(
      location=loc_list[i],
      popup = (
        "<strong>District:</strong> {District}"
          "<br>"
           "<br>"
        "<strong>Total Confirmed Cases:</strong> {Confirmed}<br>"
           "<br>"
        "<strong> Total Recovered Cases:</strong>{Recovered}<br>"
           "<br>"
        "<strong>Total Active Cases:</strong> {Active}<br>"
    ).format(District=str(merge_dfs.iloc[i]['District']), Confirmed=str(merge_dfs.iloc[i]['Confirmed']),Recovered=str(merge_dfs.iloc[i]['Recovered']),Active=str(merge_dfs.iloc[i]['Active'])),
      radius=merge_dfs.iloc[i]['Confirmed']/8,
      color='darkorange',
      fill=True,
      fill_color='darkorange'
   ).add_to(ConfirmedCasesMap)
ConfirmedCasesMap

## Total Deceased in each state till date due to Covid-19 

In [16]:
DeathsMap = folium.Map(location=[29.742316, 76.219202], tiles='OpenStreetMap', zoom_start=5)
 
# I add marker one by one on the map
for i in range(0,len(merge_dfs)):
    folium.Circle(
      location=loc_list[i],
       popup = (
            "<strong>District:</strong> {District}"
          "<br>"
           "<br>"
        "<strong>Total Deaths:</strong> {Deaths}</br>"
    ).format(Deaths=str(merge_dfs.iloc[i]['Deceased']),District=str(merge_dfs.iloc[i]['District'])),
      radius=merge_dfs.iloc[i]['Confirmed']/8,
      color='red',
      fill=True,
      fill_color='red'
   ).add_to(DeathsMap)

DeathsMap